In [1]:
!pip install plotly
!pip install textblob

     ---------------------------------------- 15.6/15.6 MB 8.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 636.8/636.8 kB 6.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from textblob import TextBlob

In [3]:
df_strm = pd.read_json('StreamingHistory.json')
df_strm

,endTime,artistName,trackName,msPlayed
0,2021-11-04 11:16,Morris the Friend,Orange Room,156973
1,2021-11-04 11:20,Tame Impala,Breathe Deeper - Lil Yachty Remix,288386
2,2021-11-04 11:24,AUDREY NUNA,Top Again (feat. Saba),187466
3,2021-11-04 11:28,Jelani Aryeh,Marigold,243113
4,2021-11-04 11:42,PHFAT,Whatever You Like,41683
...,...,...,...,...
828,2022-04-25 07:32,6LACK,Let Her Go,177171
829,2022-04-25 07:36,6LACK,Sorry,210177
830,2022-04-25 07:40,6LACK,Pretty Little Fears (feat. J. Cole),240341
831,2022-04-25 07:44,6LACK,Disconnect,248675


In [4]:

df_strm = pd.read_json('StreamingHistory.json')
df_strm['endTime'] = pd.to_datetime(df_strm['endTime'])
df_strm.set_index('endTime', inplace=True)
weekly_avg_time = df_strm.resample('W').sum()['msPlayed'] / (1000 * 60)
fig = go.Figure()
fig.add_trace(go.Scatter(x=weekly_avg_time.index, y=weekly_avg_time.values, mode='lines+markers'))
fig.update_layout(
    title='Average Time Listened per Week',
    xaxis=dict(title='Week'),
    yaxis=dict(title='Average Time Listened (minutes)'),
    template='plotly_dark'
)
fig.show()

/tmp/ipykernel_159285/2046056356.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weekly_avg_time = df_strm.resample('W').sum()['msPlayed'] / (1000 * 60)


In [5]:
df_strm = pd.read_json('StreamingHistory.json')
artist_listening_time = df_strm.groupby('artistName')['msPlayed'].sum() / (1000 * 60)
tracks_listening_time = df_strm.groupby('trackName')['msPlayed'].sum() / (1000 * 60) 
artist_listening_time = artist_listening_time.sort_values(ascending=False)
tracks_listening_time = tracks_listening_time.sort_values(ascending=False)
fig_pie = go.Figure()
fig_pie.add_trace(go.Pie(labels=artist_listening_time.index, values=artist_listening_time.values, hole=0.3, textinfo='none'))
fig_pie.update_layout(title='Listening Time Distribution by Artist')
fig_bar = go.Figure()
fig_bar.add_trace(go.Bar(x=tracks_listening_time.index, y=tracks_listening_time.values))
fig_bar.update_layout(title='Listening Time Distribution by tracks', xaxis=dict(title='tracks'), yaxis=dict(title='Listening Time (minutes)'))
fig_pie.show()
fig_bar.show()


In [6]:
df_strm = pd.read_json('StreamingHistory.json')
df_strm['endTime'] = pd.to_datetime(df_strm['endTime'])
df_strm['day_of_week'] = df_strm['endTime'].dt.dayofweek
df_strm['hour_of_day'] = df_strm['endTime'].dt.hour
listening_frequency = df_strm.groupby(['day_of_week', 'hour_of_day']).size().unstack()
listening_frequency = listening_frequency.reindex(index=[0, 1, 2, 3, 4, 5, 6])

fig_heatmap = go.Figure(data=go.Heatmap(
    z=listening_frequency.values,
    x=list(range(24)),  # Hours of the day
    y=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    colorscale='Reds',  # Color scale
))

fig_heatmap.update_layout(
    title='Listening Frequency Heatmap',
    xaxis=dict(title='Hour of the Day'),
    yaxis=dict(title='Day of the Week'),
    margin=dict(l=50, r=50, t=50, b=50),
)
fig_heatmap.show()


In [7]:
df_strm = pd.read_json('StreamingHistory.json')
def classify_sentiment(track_name):
    analysis = TextBlob(track_name)
    if analysis.sentiment.polarity > 0:
        return 'Happy'
    elif analysis.sentiment.polarity < 0:
        return 'Sad'
    else:
        return 'Neutral'

df_strm['sentiment']  = df_strm['trackName'].apply(lambda x: classify_sentiment(x))

In [8]:
df_strm['endTime'] = pd.to_datetime(df_strm['endTime'])
df_strm['hour_of_day'] = df_strm['endTime'].dt.hour
sentiment_distribution = df_strm.groupby(['hour_of_day', 'sentiment']).size().unstack().fillna(0)
fig_bar = go.Figure()
for sentiment in sentiment_distribution.columns:
    fig_bar.add_trace(go.Bar(
        x=sentiment_distribution.index,
        y=sentiment_distribution[sentiment],
        name=sentiment
    ))

fig_bar.update_layout(
    title='Sentiment Distribution by Hour of the Day',
    xaxis=dict(title='Hour of the Day'),
    yaxis=dict(title='Number of Songs'),
    barmode='stack',
)
fig_bar.show()
